In [1]:
print('Hello World')

Hello World


In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import service
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings("ignore")
import re
import time
import requests
from selenium.webdriver.common.keys import Keys

# 구조
> 스크롤             
> 요소 보이면 크롤링                   
> 스크롤 하다가 더보기 보이면 클릭

In [ ]:
chrome_options = Options()
# 전체화면
chrome_options.add_argument("--start-maximized")
# 알림 표시 제거
chrome_options.add_argument("--disable-notifications")
cour_url = '''https://www.coursera.org/search?query=aws'''
driver = webdriver.Chrome(options=chrome_options)
driver.get(cour_url)

#스크롤하며 공고 찾기

cour_res = pd.DataFrame(columns = ['강의명', '기술', '별점', '리뷰수'])

for post_num in range(1, 999):
    while True:
        try:
            # 공고 찾기
            title = driver.find_element(
                By.XPATH, f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[1]/div[2]'
                ).text
            
            # 있으면 클릭하고 break
            if title.is_displayed():
                try:
                    # 강의명
                    title = WebDriverWait(driver, 10).until(
                            Ec.presence_of_element_located(
                                (
                                By.XPATH, f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[1]/div[2]'
                            )
                            )
                            ).text
                    # 기술
                    tec = driver.find_element(By.XPATH, f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[2]').text
                    # 별점
                    star = driver.find_element(By.XPATH, f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[3]').text.split('\n')[0]
                    # 리뷰수
                    review = driver.find_element(By.XPATH, f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[3]').text.split('\n')[1]
                except:
                    None
        # 없으면 오류제거하고 while True 반복 진행
        except NoSuchElementException:
            pass
        # 스크롤 시작점=0, 500픽셀씩 내리기
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(2)
        cour_temp = pd.DataFrame([title, tec, star, review], index = ['강의명', '기술', '별점', '리뷰수'])
        cour_res = pd.concat([cour_res, cour_temp])


# 구조 2

In [109]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
# 전체화면
chrome_options.add_argument("--start-maximized")
# 알림 표시 제거
chrome_options.add_argument("--disable-notifications")
cour_url = '''https://www.coursera.org/search?query=aws'''
driver = webdriver.Chrome(options=chrome_options)
driver.get(cour_url)

cour_res = pd.DataFrame(columns = ['강의명', '기술', '별점', '리뷰수'])


# 포스트 초기
post_num = 1
# pre = 지금 스크롤 높이
pre_height = driver.execute_script('return document.body.scrollHeight')

# 스크롤 하며 Crawling , 더보기 나오면 클릭 루프 시작
while True:
    # 강의명
    title_ele = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH,  f'//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/ul[1]/li[{post_num}]/div/div/div/div/div/div[2]/div[1]'
             )
             )
             )
    if title_ele.is_displayed():
        title = title_ele.text.split('\n')[1]
        
        # 기술
        tec = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[2]').text
        
        # 별점
        star_review = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div/div/div/div/div[2]/ul/li[{post_num}]/div/div/div/div/div/div[2]/div[3]').text.split('\n')
        star = star_review[0]
        review = star_review[1]
        
        # DataFrame에 추가
        cour_temp = pd.DataFrame(
            [title, tec, star, review],
            index=['강의명', '기술', '별점', '리뷰수']
            ).T
        cour_res = pd.concat([cour_res, cour_temp])
    
    post_num += 1

    # 스크롤 시작점=0, 500픽셀씩 내리기
    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(2)
    
    # 새로운 스크롤 위치
    new_height = driver.execute_script('return document.body.scrollHeight')
    # 이전 스크롤 위치와 비교하여 같다면 페이지의 끝에 도달한 것이므로 루프 종료
    if new_height == pre_height:
        break
    pre_height = new_height

    # 더보기 버튼 나오면 누르기
    try:
        driver.find_element(
            By.XPATH,
            '//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/div[4]/button/span'
            ).is_displayed()
        driver.find_element(
            By.XPATH,
            '//*[@id="rendered-content"]/div/div/main/div[2]/div/div/div/div/div[2]/div[4]/button/span'
            ).click
    except:
        Exception

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF693C77012+3522402]
	(No symbol) [0x00007FF693898352]
	(No symbol) [0x00007FF693745ABB]
	(No symbol) [0x00007FF69378BF0E]
	(No symbol) [0x00007FF69378C08C]
	(No symbol) [0x00007FF6937CE437]
	(No symbol) [0x00007FF6937AF09F]
	(No symbol) [0x00007FF6937CBDA3]
	(No symbol) [0x00007FF6937AEE03]
	(No symbol) [0x00007FF69377F4D4]
	(No symbol) [0x00007FF6937805F1]
	GetHandleVerifier [0x00007FF693CA9B9D+3730157]
	GetHandleVerifier [0x00007FF693CFF02D+4079485]
	GetHandleVerifier [0x00007FF693CF75D3+4048163]
	GetHandleVerifier [0x00007FF6939CA649+718233]
	(No symbol) [0x00007FF6938A4A3F]
	(No symbol) [0x00007FF69389FA94]
	(No symbol) [0x00007FF69389FBC2]
	(No symbol) [0x00007FF69388F2E4]
	BaseThreadInitThunk [0x00007FFACBF426BD+29]
	RtlUserThreadStart [0x00007FFACC8EDFB8+40]


In [108]:
cour_res

,강의명,기술,별점,리뷰수
